# Fit quiescent light curve under flare with Gaussian Process regression

based on this tutorial https://celerite2.readthedocs.io/en/latest/tutorials/first/

In [ ]:
# basics
import numpy as np
import pandas as pd
from scipy.optimize import minimize, curve_fit

# matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib 
matplotlib.rc('xtick', labelsize=12) 
matplotlib.rc('ytick', labelsize=12) 

font = {'family' : 'courier',
        'weight' : 'normal',
        'size'   : 15}


# GP
import celerite2
from celerite2 import terms

# data management
from funcs.helper import read_custom_aperture_lc, fetch_lightcurve
import os
CWD = "/".join(os.getcwd().split("/")[:-2])


# model
# from funcs.multiperiod import show_flare, find_period

import time
# Create a time stamp for this run
tstamp = time.strftime("%d_%m_%Y_%H_%M", time.localtime())

# MCMC analytics
import corner
import emcee

# define global range of frequencies
freq = np.linspace(0.02, 30, 500)
omega = 2 * np.pi * freq

def plot_psd(gp):
    """Plot power spectral density of GP.
    Unit: 1/day
    Frequencies: 0.02-30 per day
    """
    for n, term in enumerate(gp.kernel.terms):
        plt.loglog(freq, term.get_psd(omega), label="term {0}".format(n + 1))
    plt.loglog(freq, gp.kernel.get_psd(omega), ":k", label="full model")
    plt.xlim(freq.min(), freq.max())
    plt.legend(frameon=False)
    plt.xlabel("frequency [1 / day]")
    plt.ylabel("power [day ppt$^2$]")
    

def log_prob(params, gp, prior_sigma=2.):
    """Log Likelihood with a normal prior around the chosen inits."""

    gp = set_params(params, gp)
    return (gp.log_likelihood(y) - 0.5 * np.sum((params / prior_sigma) ** 2),
            gp.kernel.get_psd(omega),)
  

def plot_prediction(gp):
    """Make a prediction of the GP model for the missing data."""
    plt.scatter(true_t, true_y, c="k", s=3.5, alpha=0.3, label="data")
    if gp:
        mu, variance = gp.predict(y, t=true_t, return_var=True)
        sigma = np.sqrt(variance)
        plt.plot(true_t, mu, label="prediction")
        plt.fill_between(true_t, mu - sigma, mu + sigma, color="C0", alpha=0.2)
        
        
def set_params(params, gp):
    """Set kernel parameters, the flux mean value, and a variance
    added to the diagonal of the covariance matrix.
    """
    gp.mean = params[0]
    inits = np.exp(params[1:])
    gp.kernel = (terms.RotationTerm(sigma=inits[0], period=inits[1], Q0=inits[2], dQ=inits[3], f=inits[4]) +
                 terms.SHOTerm(sigma=inits[5], rho=inits[6], tau=inits[7]))
    gp.compute(t, diag=yerr ** 2, quiet=True)
    return gp


def neg_log_like(params, gp):
    """Calculate negative log likelihood of the GP model."""
    gp = set_params(params, gp)
    return -gp.log_likelihood(y)

In [ ]:
# get stellar parameters
lcs = pd.read_csv(f"{CWD}/data/summary/lcs.csv")

In [ ]:
# select a target
target = lcs.iloc[-1]
target

In [ ]:
# get tstamp
inits = pd.read_csv(f"{CWD}/data/summary/inits_decoupled_GP.csv")
inits_ = inits[str(target.ID) == inits.ID.str.strip("ab")]#.str.strip("ab")
inits_

In [ ]:
row = inits_.iloc[1]
ID =row.ID.strip("ab")
lc = pd.read_csv(f"{CWD}/data/lcs/{row.tstamp}_{ID}.csv")
# target.view_start, target.view_stop = lc.t.iloc[0], lc.t.iloc[-1]
# assert target.view_start == lc.t.iloc[0]
# assert target.view_stop == lc.t.iloc[-1]

In [ ]:
target.view_start, target.view_stop

In [ ]:
# get TESS LC here
flc = fetch_lightcurve(target)

# select subset of LC to run GP regression on
gpstart, gpstop = target.view_start-1., target.view_stop+1.
select = np.where((flc.time > gpstart) & (flc.time < gpstop))
flc = flc[select]

# remove non-finite values
print(flc.flux.shape)
flc = flc[np.where(np.isfinite(flc.flux))]
flc.flux.shape

In [ ]:
# define true values before you start
true_t = flc.time
true_y = flc.flux
true_yerr = flc.flux_err

In [ ]:
# mask flare (tinker with the actual range if needed)
mask = np.where((flc.time > target.view_stop) | (flc.time < target.view_start))
flcm = flc[mask]

In [ ]:
# mask all positive outliers beyond 5 sigma
flcm = flcm[flcm.flux < np.median(flcm.flux) + np.std(flcm.flux) * 5]

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(true_t, true_y, alpha=.5, label="masked flare")
plt.plot(flcm.time, flcm.flux,c="k", label="used for GP prediction")
plt.xlim(gpstart,gpstop)
plt.xlabel(f"Time [Barycentric Julian Date - {target.BJDoff} days]",fontsize=15)
plt.ylabel(r"Flux [$e^{-}$ s$^{-1}$]",fontsize=15)
plt.legend(frameon=False, loc=1, fontsize=15)
plt.tight_layout()
# plt.savefig(f"{CWD}/analysis/results/rotation/{tstamp}_TIC{target.ID}_GP_lc.png", dpi=300)

In [ ]:
# show what you are feeding the GP
t = flcm.time
y = flcm.flux
yerr = flcm.flux_err
plt.errorbar(t, y, yerr=yerr, fmt=".k", capsize=0)
plt.xlabel("time")
plt.ylabel("flux")
_ = plt.title(f"TIC {target.ID}")

In [ ]:
t.shape

In [ ]:
# add rotation period as determined from all available light curves
props = pd.read_csv(f"{CWD}/data/summary/inclination_input.csv")

P = props[props["id"] == int(ID)].iloc[0].prot_d

In [ ]:
# initial values for the kernels to pass to MCMC sampler
inits = [11., P, 150., .4, .9, 2.3, 35., 10.]

# Rotational term
term1 = terms.RotationTerm(sigma=inits[0], period=P, Q0=inits[2], dQ=inits[3], f=inits[4])

# Quasi-periodic term
term2 = terms.SHOTerm(sigma=inits[5], rho=inits[6], tau=inits[7])

# define kernel as sum of background and rotational variability
kernel = term1 + term2

# Setup the GP
gp = celerite2.GaussianProcess(kernel, mean=np.nanmedian(flcm.flux))
gp.compute(t, yerr=yerr)

print("Initial log likelihood: {0}".format(gp.log_likelihood(y)))

In [ ]:
plot_psd(gp)
plt.title("Initial PSD");

In [ ]:
# Plot initial prediction
plt.figure(figsize=(14,6))
plt.title("initial prediction")
plot_prediction(gp)

plt.xlim(gpstart,gpstop)
plt.xlabel(f"Time [Barycentric Julian Date - {target.BJDoff} days]",fontsize=15)
plt.ylabel(r"Flux [$e^{-}$ s$^{-1}$]",fontsize=15)
plt.legend(frameon=False, loc=1, fontsize=15)
plt.savefig(f"{CWD}/analysis/results/rotation/{tstamp}_TIC{target.ID}_GP_init_prediction.png", dpi=300)

In [ ]:
# add mean flux value to the MCMC inits
initial_params =  [np.nanmedian(flcm.flux)] + inits
# take the logarithm
initial_params = [np.log(x) for x in initial_params]
# minimize negative log likelihood
soln = minimize(neg_log_like, initial_params, method="L-BFGS-B", args=(gp,))
# pass the best fit to the GP
opt_gp = set_params(soln.x, gp)
# result
soln

In [ ]:
plt.figure(figsize=(7,5))
plt.title("Maximum likelihood PSD")
plot_psd(opt_gp)

In [ ]:

plt.figure(figsize=(14,6))
plt.title("Maximum likelihood prediction")
plot_prediction(gp)

plt.xlim(gpstart,gpstop)
plt.xlabel(f"Time [Barycentric Julian Date - {target.BJDoff} days]",fontsize=15)
plt.ylabel(r"Flux [$e^{-}$ s$^{-1}$]",fontsize=15)
plt.legend(frameon=False, loc=1, fontsize=15)
plt.savefig(f"{CWD}/analysis/results/rotation/{tstamp}_TIC{target.ID}_GP_ML_prediction.png", dpi=300)

In [ ]:

# get logarithmic inits for MCMC
initslog = soln.x
# wiggle the inits
coords = initslog + 1e-5 * np.random.randn(32, len(initslog))
# initialize the sampler
sampler = emcee.EnsembleSampler(coords.shape[0], coords.shape[1], log_prob, args=(gp,))
# run chain for some time
state = sampler.run_mcmc(coords, 200, progress=True)

In [ ]:
# run chain for some longer time
state = sampler.run_mcmc(state, 100, progress=True)

In [ ]:
# get chain from sampler
chain = sampler.get_chain(discard=50, flat=True)
print(chain.shape[0]/32)

# Show MCMC results

In [ ]:
plt.figure(figsize=(14,6))

# plot a number of random samples from the posterior distribution
for sample in chain[np.random.randint(len(chain), size=10)]:
    gp = set_params(sample, gp)
    mu, variance = gp.predict(y, t=true_t, return_var=True)
    sigma = np.sqrt(variance)
    plt.plot(true_t, mu, c="b", alpha=.5)
    plt.fill_between(true_t, mu - sigma, mu + sigma, color="c", alpha=0.2)

# get 50th percentile as best fit value and make a prediction based on this solution
mcmc_res = [np.percentile(chain[:, i], [50]) for i in range(9)]
gp = set_params(mcmc_res, gp)
mu, variance = gp.predict(y, t=true_t, return_var=True)
# plot prediction
plt.plot(true_t, mu, c="r", alpha=.5)
# plot uncertainty
plt.fill_between(true_t, mu - sigma, mu + sigma, color="orange", alpha=0.7)

# plot the data, too
plot_prediction(None)

# layout
plt.xlim(gpstart,gpstop)
plt.ylim(np.min(flcm.flux)*.97, np.max(flcm.flux)*1.1)
plt.xlabel(f"Time [Barycentric Julian Date - {target.BJDoff} days]",fontsize=15)
plt.ylabel(r"Flux [$e^{-}$ s$^{-1}$]",fontsize=15)
plt.legend(frameon=False, loc=1, fontsize=15)
plt.tight_layout()
# plt.savefig(f"{CWD}/analysis/results/rotation/{tstamp}_TIC{target.ID}_GP_MCMC_prediction.png", dpi=300)

In [ ]:
# Plot posterior PSD
plot_psd(gp)
# plt.savefig(f"{CWD}/analysis/results/rotation/{tstamp}_TIC{target.ID}_GP_MCMC_prediction_PSD.png", dpi=300)

In [ ]:
# plot chain
fig, axes = plt.subplots(9, figsize=(10, 12), sharex=True)
msamples = sampler.get_chain(discard=50)

columns = ["median", "sigma_rotationterm", "period_rotationterm", "Q0_rotationterm",
           "dQ_rotationterm", "f_rotationterm","sigma_shoterm", "rho_shoterm", "tau_shoterm"]

for j,label in enumerate(columns):
    ax = axes[j]
    ax.plot(np.exp(msamples[:, :, j]), "k", alpha=0.3)
    ax.set_xlim(0, len(msamples))
    ax.set_ylabel(label, rotation=45)
    ax.yaxis.set_label_coords(-0.1, 0.5)
    
# plt.savefig(f"{CWD}/analysis/results/rotation/{tstamp}_TIC{target.ID}_GP_MCMC_prediction_chain.png", dpi=300)

In [ ]:
# plot corner visualization of marginalized posterior distributions
corner.corner(chain);
# plt.savefig(f"{CWD}/analysis/results/rotation/{tstamp}_TIC{target.ID}_GP_MCMC_prediction_corner.png", dpi=300);

In [ ]:
# save results to file
resmcmc = pd.DataFrame(data=np.exp(chain), columns=columns)
# resmcmc.to_csv(f"{CWD}/analysis/results/rotation/{tstamp}_TIC{target.ID}_GP_MCMC_chain.csv", index=False)

# Save inits and plot new light curve

In [ ]:
# new median
# new tstamp

# optional:
# extramask = np.where((flc.time > target.view_stop-.3) | (flc.time < target.view_start-.15))

# select the part of the light curve that you want to you for modulation model fitting
selectformcmc = (true_t > target.view_start) & (true_t <= target.view_stop)

# define new flux, time, and error arrays
modelflux = mu[selectformcmc]
modelflux_err = sigma[selectformcmc]
fullflux = true_y[selectformcmc]
fullyerr = true_yerr[selectformcmc]
time = true_t[selectformcmc]

## Plot the newly defined light curve

In [ ]:
plt.figure(figsize=(7,7))

# un-detrended flux
plt.scatter(time, fullflux, s=1, c="c", 
            label="PDCSAP_FLUX")

# quiescent flux
plt.plot(time, modelflux, label="GP prediction")

# flare-only flux
plt.plot(time, fullflux-modelflux + np.nanmedian(modelflux), 
         c="k", linewidth=.5,
         label="flux - GP prediction + median")

# median
plt.plot(time, [np.nanmedian(modelflux)] * len(time), 
         linestyle="dotted", c="grey", label="median")

# layout
plt.xlim(target.view_start,target.view_stop)
plt.ylim(np.min(flcm.flux)*.97, np.max(flcm.flux)*1.4)
plt.xlabel(f"Time [Barycentric Julian Date - {target.BJDoff} days]",fontsize=15)
plt.ylabel(r"Flux [$e^{-}$ s$^{-1}$]",fontsize=15)
plt.legend(frameon=False, loc=2, fontsize=14)
plt.tight_layout()


# plt.savefig(f"{CWD}/analysis/results/rotation/{tstamp}_TIC{target.ID}_GP_MCMC_new_lc.png", dpi=300)

In [ ]:
# co-add PDCSAP and GP errors
new_flux_err = np.sqrt(fullyerr**2 + modelflux_err**2)

# Define Dataframe with new light curve
new_t = time
new_phi = (new_t - new_t[0]) / P * 2 * np.pi
new_median = np.nanmedian(modelflux)
new_flux = fullflux - modelflux + np.nanmedian(modelflux)


newlc = pd.DataFrame({'phi': new_phi,
                      'flux': new_flux,
                      't':new_t, 
                      'flux_err':new_flux_err,
                      'median_' : new_median, 
                      "modelflux": modelflux, 
                      "modelflux_err": modelflux_err,})


In [ ]:
# check new time stamp and median value
tstamp, new_median

In [ ]:
newlc.to_csv(f"{CWD}/data/lcs/{tstamp}_{target.ID}.csv", index=False)

# Save new light curve inits for modulation model fitting if you had defined them previously

In [ ]:
row.tstamp, row.median

In [ ]:
row.tstamp = tstamp
row.median = new_median

In [ ]:
row.tstamp, row.median

In [ ]:
inits = pd.read_csv(f"{CWD}/data/summary/inits_decoupled_GP.csv")

inits = inits.append(row)

inits.to_csv(f"{CWD}/data/summary/inits_decoupled_GP.csv", index=False)